# Máxima profundidad de árbol de decisión mediante validación cruzada

In [71]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
dataset = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.2, random_state=42)

Calculo del error de un árbol sin limitar la profundidad:

In [73]:
tree_no_limit = DecisionTreeRegressor(random_state=42)
mae_no_limit = -cross_val_score(tree_no_limit, X_train, y_train,
                            scoring = "neg_mean_absolute_error",
                            cv = 10).mean().round(2)
mae_no_limit

65.78

Uso un diccionario de modelos con diferentes profundidades máximas y otro con los errores medios absolutos obtenidos en la validación cruzada.

Los inicializo con el modelo sin limitación de profundidad y la clave None (la misma que usa por defecto el parámetro `max_depth` de `DecisionTreeRegressor` para no limitar).

In [74]:
tree_depth = {None: tree_no_limit}
maes_depth = {None: mae_no_limit}

for i in range(1, 10):
    tree_depth[i] = DecisionTreeRegressor(random_state=42, max_depth=i)
    maes_depth[i] = -cross_val_score(tree_depth[i], X_train, y_train,
                              scoring = "neg_mean_absolute_error").mean().round()

In [75]:
for depth, mae in maes_depth.items():
    print(f"Profundidad {depth} - MAE: {mae}")

depth_min_mae = min(maes_depth, key=maes_depth.get)
print(f"La mejor profundidad es {depth_min_mae} con un MAE de {maes_depth[depth_min_mae]}")

Profundidad None - MAE: 65.78
Profundidad 1 - MAE: 56.0
Profundidad 2 - MAE: 50.0
Profundidad 3 - MAE: 52.0
Profundidad 4 - MAE: 53.0
Profundidad 5 - MAE: 54.0
Profundidad 6 - MAE: 56.0
Profundidad 7 - MAE: 59.0
Profundidad 8 - MAE: 60.0
Profundidad 9 - MAE: 61.0
La mejor profundidad es 2 con un MAE de 50.0


Vemos que entre los valores arbitrariamente elegidos de profundidad [1-10] encontramos un *mínimo de la función de coste en profundidad 2*, incrementándose consistentemente en los siguientes y siendo inferior también que un árbol sin límite. Parece por tanto razonable asumir que no haya otros mínimos en profundidades superiores.

Fijaremos por tanto el **hiperparámetro** de profundidad en 2.

Definidos los hiperparámetros, entrenamos el modelo con el conjunto de entrenamiento completo y **evaluamos su rendimiento con el conjunto de test, que no se ha usado en ningún entrenamiento y por tanto no ha sesgado la elección de hiperparámetros**.

In [76]:
from sklearn.metrics import mean_absolute_error

model = tree_depth[depth_min_mae].fit(X_train, y_train)
y_pred = model.predict(X_test)
mae_test = mean_absolute_error(y_test, y_pred)
print(" MAE(test):", mae_test.round(2))

 MAE(test): 49.37


## Alternativa: elección del hiperparámetro utilizando GridSearchCV

In [79]:
from sklearn.model_selection import GridSearchCV

tree = DecisionTreeRegressor(random_state=42)

params = {
    'max_depth': list(range(1, 10)),
}
grid_search_cv = GridSearchCV(tree, params, cv=10,
                              scoring="neg_mean_absolute_error")

grid_search_cv.fit(dataset.data, dataset.target)
print(grid_search_cv.best_estimator_)

DecisionTreeRegressor(max_depth=2, random_state=42)


<!-- TODO: ## Regresión polinómica -->